# Notebook Used To Generate and Submit Scripts

In [14]:
import os

In [15]:
new_exp_scripts_dir = '../../las_sim_tkt_new_exp_scripts'
if not os.path.exists(new_exp_scripts_dir):
    os.makedirs(new_exp_scripts_dir)

## Generate exp_job.sh

In [16]:
exp_job_script_template = './exp_job_tmpl.sh'
new_exp_job_script = os.path.join(new_exp_scripts_dir, 'exp_job.sh')

In [60]:
sbatch_account = 'def-rgorbet'
sbatch_gpu = '1'
sbatch_nodes = '1'
sbatch_ntasks = '4'
sbatch_mem_per_cpu = '1024M'
sbatch_time_d = 0
sbatch_time_h = 10
sbatch_time_m = 0

exp_run_time = sbatch_time_d*24*60 + sbatch_time_h*60 + sbatch_time_m

las_sim_tkt_dir = '/scratch/lingheng/las_sim_tkt'
las_sim_tkt_pkg_dir = '/scratch/lingheng/las_sim_tkt_pkg'    # saving packages for offline installation
las_sim_tkt_dep_dir = '/scratch/lingheng/las_sim_tkt_dep'    # saving installed shared softwares such as Processing, Node, Miniconda, Mujoco
las_sim_tkt_data_dir = '/scratch/lingheng/las_sim_tkt_data'  # saving experiment data

nv_driver_version='470.103.01'  # Adapt deriver version to your system: Graham: 470.103.01, Cedar: 510.47.03

if (sbatch_gpu == '0' and nv_driver_version != '') or (sbatch_gpu != '0' and nv_driver_version == ''):
    raise Warning("gpu is allocated, but nv_driver_version is not specified!")

if sbatch_gpu == '0':
    sbatch_gpu=''
else:
    sbatch_gpu='#SBATCH --gres=gpu:{0}             # Number of GPU(s) per node\n'.format(sbatch_gpu)

In [61]:
sbatch_gpu

'#SBATCH --gres=gpu:1             # Number of GPU(s) per node\n'

In [62]:
# Read exp_job.sh to extract the part that does not need to be changed.
with open(exp_job_script_template, encoding='utf8') as exp_job_sh_template_file:
    exp_job_sh_content = exp_job_sh_template_file.readlines()
    

In [63]:
# Use '/n' as newline to format the script to UNIX. Otherwise, it will be DOS fromat '\r\n'
with open(new_exp_job_script, 'w', encoding='utf8', newline='\n') as new_exp_job_sh_file:
    # Write the first 30 lines of the new job script, which needs mild modification according to the job requirements
    new_exp_job_sh_file.write('''\
#!/bin/bash
#SBATCH --account={0}
{1}#SBATCH --nodes={2}                # When requesting GPUs per node the number of nodes must be specified.
#SBATCH --ntasks={3}               # number of MPI processes
#SBATCH --mem-per-cpu={4}          # memory; default unit is megabytes
#SBATCH --time={5}-{6}:{7}         # time (DD-HH:MM)

# Define exp_run_time, which will be used by sleep command in exp_run.sh. Make sure to set it to the time 
# deducted by the time needed for all operations before running the experiment in order to neatly kill 
# the processes running different modules. For example, if the time for the job is 12 hours and the estimated 
# all other operations is 10 minutes, then we should set exp_run_time=(12*60-10)m
exp_run_time="{8}m"

# Setup related directories
las_sim_tkt_dir={9}
las_sim_tkt_pkg_dir={10}    # saving packages for offline installation
las_sim_tkt_dep_dir={11}    # saving installed shared softwares such as Processing, Node, Miniconda, Mujoco
las_sim_tkt_data_dir={12}  # saving experiment data

# Setup nvidia GPU driver version. To find the driver version, on a compute node, use nvidia-smi to query gpu deriver version:
#   output_nv_driver_version=($(nvidia-smi --query-gpu=driver_version --format=csv))    # Turn return as array with ()
#   nv_driver_version=${{output_nv_driver_version[1]}}    # Retrieve driver version
nv_driver_version={13}             # Graham: 470.103.01, Cedar: 510.47.03
if [ -z "${{nv_driver_version}}" ]; then
    echo "$nv_driver_version is unset or set to the empty string"
    add_nvdriver=false
else
    add_nvdriver=true
fi
'''.format(sbatch_account, sbatch_gpu, sbatch_nodes, sbatch_ntasks, sbatch_mem_per_cpu, 
           sbatch_time_d, sbatch_time_h, sbatch_time_m, exp_run_time, 
           las_sim_tkt_dir, las_sim_tkt_pkg_dir, las_sim_tkt_dep_dir, las_sim_tkt_data_dir, nv_driver_version))
    
    # Write the rest of the exp_job.sh to new job script
    for sh_line in exp_job_sh_content[33:]:
        new_exp_job_sh_file.write(sh_line)

## Generate exp_run.sh

In [37]:
exp_run_script_template = './exp_run_tmpl.sh'
new_exp_run_script = os.path.join(new_exp_scripts_dir, 'exp_run.sh')

In [38]:
# Read exp_job.sh to extract the part that does not need to be changed.
with open(exp_run_script_template, encoding='utf8') as exp_run_script_template_file:
    exp_run_sh_content = exp_run_script_template_file.readlines()
    

In [41]:
# Use '/n' as newline to format the script to UNIX. Otherwise, it will be DOS fromat '\r\n'
with open(new_exp_run_script, 'w', encoding='utf8', newline='\n') as new_exp_run_sh_file:
    # Write the first part of unchanged lines of the new job script
    for run_line in exp_run_sh_content[:68]:
        new_exp_run_sh_file.write(run_line)
    
    # Write the part need to be modified
    new_exp_run_sh_file.write('''\
# Run python script （Note: this is the part need to be changed for different experiment runs.）
if [ -d "$exp_run_data_dir/PL-Teaching-Data" ]; then
  nohup python $exp_run_code_dir/PL-POMDP/pl/teach.py --resume_exp_dir $exp_run_data_dir/PL-Teaching-Data/exp_name &>$exp_run_data_dir/console_python.out &
else
  nohup python $exp_run_code_dir/PL-POMDP/pl/teach.py --env_id LAS-Meander --rl_reward_type hc_reward --rl_agent PPO &>$exp_run_data_dir/console_python.out &
fi
pid_python=$!    # Save PID for later use
''')
    
    # Write the rest lines of the new job script, which needs mild modification according to the job requirements
    for run_line in exp_run_sh_content[75:]:
        new_exp_run_sh_file.write(run_line)

## Submit Job Scripts

In [13]:
job_list = os.listdir(new_job_scripts_dir)

NameError: name 'new_job_scripts_dir' is not defined

In [ ]:
for job_sh in job_list:
    if 'job' in job_sh:
        
        os.system('bash {} '.format(os.path.join(new_job_scripts_dir, job_sh)))

In [ ]:
os.path.join(new_exp_scripts_dir)

## Submit Resumed Job Scripts